In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import time
from selenium.common.exceptions import WebDriverException

In [ ]:
driver = webdriver.Chrome()  # Or webdriver.Firefox(), depending on your browser

# Open the website, it has to be within each forum because the next button has a different structure on each forum page
driver.get('https://steamcommunity.com/discussions/forum/30/')

while True:
    topics = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'forum_topic_overlay')))

    for j in range(len(topics)):
        topics = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'forum_topic_overlay')))
        topic = topics[j]

        parent = topic.find_element(By.XPATH, './..')

        WebDriverWait(parent, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'forum_topic_icon')))
        topic_icon_div = parent.find_element(By.CLASS_NAME, 'forum_topic_icon')
        if topic_icon_div.get_attribute('title') == "This thread is pinned." :
            continue
        elif topic_icon_div.get_attribute('title') == "This thread is locked." :
            continue  

        topic.click()
        time.sleep(2)

        initial_post_date_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'date')))
        date_str = initial_post_date_element.get_attribute('title').split("@")[0].strip()

        date = datetime.strptime(date_str, '%B %d, %Y') 

        six_months_ago = datetime.now() - timedelta(days=182)

        if date < six_months_ago:
            driver.back() 
            time.sleep(2)
            continue

        # Parse the first page of comments
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comments = soup.find_all('div', {'class': 'commentthread_comment_text'})

        for comment in comments:
            comment_content = comment.text.strip()

            if 50 < len(comment_content) < 3000:
                df = pd.DataFrame([{'comment': comment_content}])
                df.to_csv('Test.csv', mode='a', header=False, index=False)

        # Go back to the list of topics
        driver.back() 
        time.sleep(2)

    # Go to the next page of topics
    try:
        next_topic_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'forum_General_4009259_30_pagebtn_next')))
        next_topic_page.click()
        time.sleep(2)
    except WebDriverException:
        break  

driver.quit()  # Close the driver when done
